In [4]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import gym

In [8]:
env = gym.make('Pendulum-v1').unwrapped

In [10]:
state_shape = env.observation_space.shape[0]
action_shpae = env.action_space.shape[0]
action_bound = [env.action_space.low, env.action_space.high]
epsilon = 0.2

In [ ]:
class PPO(object):
    def __init__(self):
        # start the tensorFlow session
        self.sess = tf.Session()
        # Define the placeholder for the state
        self.state_ph = tf.placeholder(tf.float32, [None, state_shape],'state')
        # build the value network that returns the value of a state
        with tf.variable_scope('value'):

            layer1 = tf.layers.dense(self.state_ph, 100, tf.nn.relu)
            self.v = tf.layers.dense(layer1, 1)
            self.Q = tf.placeholder(tf.float32, [None,1], 'discounted_r')